In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests_html import AsyncHTMLSession
import asyncio
import nest_asyncio
import os


In [23]:
base_url = "https://data.orc.org/public/Wpub.dll"
base_country_url = "https://data.orc.org/public/WPub.dll?action=activecerts&CountryId=" 
base_boat_url = "https://data.orc.org" 

### getting names of countries

In [39]:
response = requests.get(base_url)
country_list = []

soup = BeautifulSoup(response.text, 'html.parser')
for countryId in soup.find_all('countryid'):
    country_list.append(countryId.text)

print(country_list)


['ARG', 'ARG', 'AUS', 'AUS', 'AUT', 'AUT', 'BRA', 'BRA', 'BUL', 'BUL', 'CAN', 'CAN', 'CRO', 'CRO', 'CYP', 'CYP', 'DEN', 'DEN', 'ECU', 'ECU', 'ESP', 'ESP', 'EST', 'EST', 'FIN', 'FIN', 'FRA', 'FRA', 'GBR', 'GBR', 'GER', 'GER', 'GRE', 'GRE', 'HKG', 'HKG', 'HUN', 'HUN', 'ISR', 'ISR', 'ITA', 'ITA', 'JPN', 'JPN', 'KOR', 'KOR', 'LAT', 'LAT', 'LTU', 'LTU', 'MLT', 'MLT', 'MNE', 'MNE', 'NED', 'NED', 'NOR', 'NOR', 'PER', 'PER', 'POL', 'POL', 'POR', 'POR', 'ROU', 'ROU', 'RSA', 'RSA', 'SLO', 'SLO', 'SUI', 'SUI', 'SWE', 'SWE', 'TUR', 'TUR', 'USA', 'USA']


/home/kuba/miniconda3/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


### createng function for process contry 

In [25]:
nest_asyncio.apply()
async def get_ref_links(url):
    session = AsyncHTMLSession()
    response = await session.get(url)
    await response.html.arender()

    links = []
    refno_elements = response.html.find('a[href*="/public/WPub.dll/CC/"]')
    for elem in refno_elements:
        links.append(elem.attrs['href'])

    return links

def process_country(country_name):
    country_url = base_country_url + country_name
    links = asyncio.run(get_ref_links(country_url))
    return links

### process one boat

In [26]:
def clean_text(text):
    return text.replace('\xa0', ' ').replace('Â', '').strip()

def process_boat(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    data = {}

    # Ekstrakcja sekcji BOAT
    boat_section = soup.find('span', string="BOAT").find_next('div', class_='tabular2')
    boat_labels = boat_section.find_all('span', class_='label')
    boat_data = boat_section.find_all('span', class_='data')
    
    boat_info = {}
    for label, value in zip(boat_labels, boat_data):
        boat_info[clean_text(label.text)] = clean_text(value.text)
    
    data.update(boat_info)

    # Ekstrakcja sekcji HULL
    hull_section = soup.find('span', string="HULL").find_next('div', class_='tabular2')
    hull_labels = hull_section.find_all('span', class_='label')
    hull_data = hull_section.find_all('span', class_='data')

    hull_info = {}
    for label, value in zip(hull_labels, hull_data):
        hull_info[clean_text(label.text)] = clean_text(value.text)

    data.update(hull_info)
    
    #Ekstrakcja sekcji sail areas
    sail_info = {} 
    sail_section = soup.find('span', string="SAIL AREAS").find_next('div', class_='tabular3')
    
    for tag in sail_section:
        if 'label' in tag.get('class', []):
            current_label = tag.get_text(strip=True)
        elif 'data' in tag.get('class', []):
            if current_label:
                if 'rated' in tag.get('class', []):
                    key = f"{current_label} rated"
                else:
                    key = f"{current_label} measured"
                sail_info[key] = tag.get_text(strip=True)
    
    
    data.update(sail_info)
    
    # ekstrakcja sekcji RIG
    rig_info = {}
    rig_section = soup.find('span', string="RIG")
    if rig_section:
        rig_section = rig_section.find_next('div', class_='datacol first')
        
        for tag in rig_section:
            if 'label' in tag.get('class', []):
                current_label = tag.get_text(strip=True)
            else:
                rig_info[current_label] = tag.get_text(strip=True)
                
        rig_section = rig_section.find_next('div', class_='datacol')
        for tag in rig_section:
            if 'label' in tag.get('class', []):
                current_label = tag.get_text(strip=True)
            else:
                rig_info[current_label] = tag.get_text(strip=True)
                
        rig_section = rig_section.find_next('div', class_='datacol')
        for tag in rig_section:
            if 'label' in tag.get('class', []):
                current_label = tag.get_text(strip=True)
            else:
                rig_info[current_label] = tag.get_text(strip=True)
    
            
    data.update(rig_info)
    
    # ekstrakcja sekcji speed
    speed_info = {} 
    table = soup.find('table', class_='boatspeeds')
    headers = [header.get_text(strip=True) for header in table.find_all('tr')[0].find_all('td')]
    
    rows = table.find_all('tr')[1:]
    for row in rows:
        cells = row.find_all('td')
        row_header = cells[0].get_text(strip=True)
        row_data = [cell.get_text(strip=True) for cell in cells[1:]]
        speed_info[row_header] = dict(zip(headers[1:], row_data))
    
    data.update(speed_info)
    
    return data

In [27]:
links = process_country("GER")
data_dict = process_boat(base_boat_url + links[0])
data_dict


{'Class': 'J-125',
 'Designer': 'JOHNSTONE',
 'Builder': 'TPI',
 'Age date': '01/1998',
 'Series date': '01/1998',
 'Offset file': 'J125.off',
 'Data file': 'GER6200',
 'Length Overall': '12.527 m',
 'Maximum Beam': '3.208 m',
 'Draft': '2.480 m',
 'Displacement': '4,302 kg',
 'DLR': '3.0286',
 'IMS Division': 'Cruiser/Racer',
 'Dynamic Allowance': '0.050%',
 'Age Allowance': '0.487%',
 'Mainsail measured': '47.31',
 'Mainsail rated': '48.80',
 'Headsail Luffed measured': '35.28',
 'Headsail Luffed rated': '35.28',
 'Asymmetric measured': '153.67',
 'Asymmetric rated': '153.67',
 'Beat Angles': {'6 kt': '43.1°',
  '8 kt': '41.0°',
  '10 kt': '38.7°',
  '12 kt': '37.6°',
  '14 kt': '37.2°',
  '16 kt': '36.9°',
  '20 kt': '36.6°',
  '24 kt': '37.8°'},
 'Beat VMG': {'6 kt': '4.07',
  '8 kt': '5.01',
  '10 kt': '5.53',
  '12 kt': '5.76',
  '14 kt': '5.90',
  '16 kt': '6.00',
  '20 kt': '6.10',
  '24 kt': '6.02'},
 '52°': {'6 kt': '6.20',
  '8 kt': '7.39',
  '10 kt': '7.89',
  '12 kt': '8.1

### making database

In [28]:
def flatten_dict(d):
    flattened = {}
    for key, value in d.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flattened[f"{key} {sub_key}"] = sub_value
        else:
            flattened[key] = value
    return flattened

In [29]:
for country in country_list:
    if os.path.exists(f'output/{country}.xlsx'):
        continue
        
    i = 0
    links = process_country(country)
    df = pd.DataFrame()
    for link in links:
        data_dict = process_boat(base_boat_url + link)
        flattened_dict = flatten_dict(data_dict)
        df = pd.concat([df, pd.DataFrame([flattened_dict])], sort=False, ignore_index=True)
        i += 1
        print(country, i)
        
    df.to_excel(f'output/{country}.xlsx', index=False)
    

GER 1
GER 2
GER 3
GER 4
GER 5
GER 6
GER 7
GER 8
GER 9
GER 10
GER 11
GER 12
GER 13
GER 14
GER 15
GER 16
GER 17
GER 18
GER 19
GER 20
GER 21
GER 22
GER 23
GER 24
GER 25
GER 26
GER 27
GER 28
GER 29
GER 30
GER 31
GER 32
GER 33
GER 34
GER 35
GER 36
GER 37
GER 38
GER 39
GER 40
GER 41
GER 42
GER 43
GER 44
GER 45
GER 46
GER 47
GER 48
GER 49
GER 50
GER 51
GER 52
GER 53
GER 54
GER 55
GER 56
GER 57
GER 58
GER 59
GER 60
GER 61
GER 62
GER 63
GER 64
GER 65
GER 66
GER 67
GER 68
GER 69
GER 70
GER 71
GER 72
GER 73
GER 74
GER 75
GER 76
GER 77
GER 78
GER 79
GER 80
GER 81
GER 82
GER 83
GER 84
GER 85
GER 86
GER 87
GER 88
GER 89
GER 90
GER 91
GER 92
GER 93
GER 94
GER 95
GER 96
GER 97
GER 98
GER 99
GER 100
GER 101
GER 102
GER 103
GER 104
GER 105
GER 106
GER 107
GER 108
GER 109
GER 110
GER 111
GER 112
GER 113
GER 114
GER 115
GER 116
GER 117
GER 118
GER 119
GER 120
GER 121
GER 122
GER 123
GER 124
GER 125
GER 126
GER 127
GER 128
GER 129
GER 130
GER 131
GER 132
GER 133
GER 134
GER 135
GER 136
GER 137
GER 138
GER 

In [36]:
df
df[df['Forestay tension'].notna()]

,Class,Designer,Builder,Age date,Series date,Offset file,Data file,Length Overall,Maximum Beam,Draft,...,TPS,BD,MWT,MCG,Mizzen measured,Mizzen rated,Mizzen Staysail measured,Mizzen Staysail rated,Quad. Mainsail measured,Quad. Mainsail rated
2,X-43,JEPPESEN,X-YACHTS,11/2005,01/2003,X43STD.OFF,GER7143,12.903 m,3.954 m,2.196 m,...,,0.233,271.20,5.847,NaN,NaN,NaN,NaN,NaN,NaN
3,LUFFE 43/97,JOERGENSEN,LUFFE YACHTS,01/1997,01/1994,Lufffe43-GER4664.off,GER4664,12.993 m,3.702 m,2.119 m,...,4.590,0.218,165.00,5.230,NaN,NaN,NaN,NaN,NaN,NaN
8,X-35,JEPPESEN,X-YACHTS,01/2007,09/2005,X35.OD.OFF,GER7278,10.600 m,3.250 m,2.162 m,...,,0.220,163.20,4.440,NaN,NaN,NaN,NaN,NaN,NaN
15,SOLARIS 50 SK,SOTO,SE.RI.GI.,05/2017,01/2015,SOL50260.OFF,GER8354,15.380 m,4.534 m,2.604 m,...,7.100,0.530,,,NaN,NaN,NaN,NaN,NaN,NaN
20,COMET 38 S,VALLICELLI ANDREA,COMAR YACHTS,01/2008,12/2007,COM38S.OFF,GER7607,11.390 m,3.706 m,2.046 m,...,,0.152,181.50,5.180,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,X-382,JEPPESEN,X-YACHTS,01/2000,01/1993,X382I.OFF,GER6949,11.475 m,3.694 m,2.078 m,...,0.000,0.240,203.00,4.960,NaN,NaN,NaN,NaN,NaN,NaN
536,XP-44,JEPPESEN,X-YACHTS,05/2011,12/2010,XP-44.off,GER6544,13.289 m,4.052 m,2.689 m,...,5.190,0.300,241.30,6.250,NaN,NaN,NaN,NaN,NaN,NaN
537,X-99,NIELS JEPPESEN,X-YACHTS,01/1990,04/1985,X99.OD,X99375,9.960 m,2.990 m,1.788 m,...,,0.140,77.60,3.950,NaN,NaN,NaN,NaN,NaN,NaN
538,X-99,NIELS JEPPESEN,X-YACHTS,01/1990,04/1985,X99.OD+.OFF,X99375,9.960 m,2.990 m,1.788 m,...,,0.140,77.60,3.950,NaN,NaN,NaN,NaN,NaN,NaN
